In [ ]:
# NHAT TUNG LE, ID 426246
# LAB QUEST - 3

In [4]:
# install pyspark
!pip install pyspark

Defaulting to user installation because normal site-packages is not writeable
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845525 sha256=3d76fa9350c366fde3160a8d0d234a6abf9c6290747ba26dc2437d995c1ceeb3
  Stored in directory: c:\users\windows\appdata\local\pip\cache\wheels\51\c8\18\298a4ced8ebb3ab8a7d26a7198c0cc7035abb906bde94a4c4b
Successfully built pyspark


In [5]:
import os
from pyspark import SparkContext
from pyspark.sql import SQLContext, SparkSession 

In [3]:
# Change oath
path = "C:/Users/Windows/Documents/GitHub/movielens/ml-20m/ml-20m"
os.chdir(path)
os.listdir()

['genome-scores.csv',
 'genome-tags.csv',
 'links.csv',
 'movies.csv',
 'ratings.csv',
 'README.txt',
 'tags.csv']

In [6]:
# create spark
spark = SparkSession.builder.master('local[*]').appName("Movie Lens Recommendation").getOrCreate()

+-------+--------------------+--------------------+
|movieId|               title|              genres|
+-------+--------------------+--------------------+
|      1|    Toy Story (1995)|Adventure|Animati...|
|      2|      Jumanji (1995)|Adventure|Childre...|
|      3|Grumpier Old Men ...|      Comedy|Romance|
|      4|Waiting to Exhale...|Comedy|Drama|Romance|
|      5|Father of the Bri...|              Comedy|
|      6|         Heat (1995)|Action|Crime|Thri...|
|      7|      Sabrina (1995)|      Comedy|Romance|
|      8| Tom and Huck (1995)|  Adventure|Children|
|      9| Sudden Death (1995)|              Action|
|     10|    GoldenEye (1995)|Action|Adventure|...|
|     11|American Presiden...|Comedy|Drama|Romance|
|     12|Dracula: Dead and...|       Comedy|Horror|
|     13|        Balto (1995)|Adventure|Animati...|
|     14|        Nixon (1995)|               Drama|
|     15|Cutthroat Island ...|Action|Adventure|...|
|     16|       Casino (1995)|         Crime|Drama|
|     17|Sen

In [ ]:
# check shcema

## genome-scores
genome_scores_df = spark.read.csv("genome-scores.csv", sep=",", header = True, inferSchema=True, nullValue="NA")
genome_scores_df.show() 
genome_scores_df.printSchema()

## genome-tags
genome_tags_df = spark.read.csv("genome-tags.csv", sep=",", header = True, inferSchema=True, nullValue="NA")
genome_tags_df.show() 
genome_tags_df.printSchema()

## links
links_df = spark.read.csv("links.csv", sep=",", header = True, inferSchema=True, nullValue="NA")
links_df.show() 
links_df.printSchema()

## movies
movies_df = spark.read.csv("movies.csv", sep=",", header = True, inferSchema=True, nullValue="NA")
movies_df.show() 
movies_df.printSchema()

## ratings
ratings_df = spark.read.csv("ratings.csv", sep=",", header = True, inferSchema=True, nullValue="NA")
ratings_df.show() 
ratings_df.printSchema()

## tags
tags_df = spark.read.csv("tags.csv", sep=",", header = True, inferSchema=True, nullValue="NA")
tags_df.show() 
tags_df.printSchema()

+-------+-----+--------------------+
|movieId|tagId|           relevance|
+-------+-----+--------------------+
|      1|    1|0.025000000000000022|
|      1|    2|0.025000000000000022|
|      1|    3|0.057750000000000024|
|      1|    4|             0.09675|
|      1|    5|             0.14675|
|      1|    6| 0.21700000000000003|
|      1|    7|               0.067|
|      1|    8| 0.26275000000000004|
|      1|    9|               0.262|
|      1|   10| 0.03199999999999997|
|      1|   11|               0.577|
|      1|   12| 0.11625000000000002|
|      1|   13|               0.188|
|      1|   14|0.008000000000000007|
|      1|   15|0.036750000000000005|
|      1|   16| 0.28174999999999994|
|      1|   17|0.007000000000000006|
|      1|   18| 0.11049999999999999|
|      1|   19|              0.6705|
|      1|   20|              0.1845|
+-------+-----+--------------------+
only showing top 20 rows

root
 |-- movieId: integer (nullable = true)
 |-- tagId: integer (nullable = true)
 |-